In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

In [ ]:
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
student_data.head()

In [ ]:
#area-summary
Total_School = school_data['school_name'].nunique()
Total_Students = student_data['Student ID'].nunique()
Total_Budget = school_data['budget'].sum()
Average_math_score = student_data["maths_score"].mean()
Average_reading_score = student_data["reading_score"].mean()
Math_over_50 = student_data.loc[student_data["maths_score"]>= 50]
Total_pass = len(Math_over_50)
Percent_pass_math = (Total_pass / Total_Students)*100

Reading_over_50 = student_data.loc[student_data["reading_score"]>= 50]
Total_pass_r = len(Reading_over_50)
Percent_pass_reading = (Total_pass_r / Total_Students)*100

Overall_pass = student_data.loc[(student_data["maths_score"]>=50) & (student_data["reading_score"]>= 50)
    , :]

Percentage_total_pass_df = (len(Overall_pass) /Total_Students)*100

area_summary = pd.DataFrame([{"Total School": Total_School, "Total Students":Total_Students, "Total Budget":Total_Budget, "Average maths score": Average_math_score, "Average reading score":Average_reading_score,"% passing maths":Percent_pass_math,"% passing reading":Percent_pass_reading,"% overall passing":Percentage_total_pass_df}])  
area_summary

In [ ]:
# Combine the data into a single dataset.  
school_data_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [ ]:
#Byschool

per_school_type = school_data.groupby(["school_name"])[("type")].size()
per_school_tstudent = student_data.groupby(["school_name"])[("student_name")].size()
per_school_tbudget = school_data.groupby(["school_name"])[("budget")].sum()
budget_by_student = per_school_tbudget / per_school_tstudent
per_school_aveg_math = school_data_df.groupby(["school_name"])[("maths_score")].mean()
per_school_aveg_reading = school_data_df.groupby(["school_name"])[("reading_score")].mean()

per_school_math_over50 = Math_over_50.groupby(["school_name"]).size()
Percent_pass_math_byS = (per_school_math_over50 /per_school_tstudent)*100

per_school_reading_over50 = Reading_over_50.groupby(["school_name"]).size()
Percent_pass_math_per_school = (per_school_reading_over50 /per_school_tstudent)*100
Percent_pass_reading_byS = (Total_pass_r / Total_Students)*100 

per_school_overall_pass_byS = Overall_pass.groupby(["school_name"]).size()
Percentage_total_pass_byS = (per_school_overall_pass_byS /per_school_tstudent)*100

school_type = school_data.groupby(["school_name"])[("type")].max()

d = {"School Type": school_type,"Total Students": per_school_tstudent,"Total School Budget":per_school_tbudget,"Per Student Budget":budget_by_student, "Average maths score": per_school_aveg_math, "Average reading score": per_school_aveg_reading, "%Passing Maths":Percent_pass_math_byS, "%Passing Reading":Percent_pass_reading_byS,"% Overall Passing":Percentage_total_pass_byS}
df2 = pd.DataFrame(data=d) 
per_school_summary = pd.DataFrame(data=d).style.format({
  "Total School Budget": lambda val: f'${val:,.2f}', "Per Student Budget": lambda val: f'${val:,.2f}'
})

per_school_summary

In [ ]:
# top_schools
df2_sorted = (df2.sort_values("% Overall Passing", ascending=False)).head(5)
df2_sorted

top_schools = pd.DataFrame(df2_sorted).style.format({
  "Total School Budget": lambda val: f'${val:,.2f}', "Per Student Budget": lambda val: f'${val:,.2f}'})
top_schools

In [ ]:
# Reset Index
df2_sorted = df2_sorted.reset_index(drop=True)
#health_expenditure_sorted_df.head()

In [ ]:
#bottom_schools
df2_sorted_b = (df2.sort_values("% Overall Passing", ascending=True)).head(5)
df2_sorted_b
bottom_schools = pd.DataFrame(df2_sorted_b).style.format({
  "Total School Budget": lambda val: f'${val:,.2f}', "Per Student Budget": lambda val: f'${val:,.2f}'})
bottom_schools

In [ ]:
#math_score_by_year
aveg_maths_by_year_by_school = student_data.groupby(["school_name","year"])["maths_score"].mean()
aveg_maths_by_year_by_school

math_score_by_year = pd.DataFrame(aveg_maths_by_year_by_school)
math_score_by_year


In [ ]:
#reading_score_by_year
aveg_reading_by_year_by_school = student_data.groupby(["school_name","year"])["reading_score"].mean()
aveg_reading_by_year_by_school

reading_score_by_year = pd.DataFrame(aveg_reading_by_year_by_school)
reading_score_by_year

In [ ]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
pd.cut(df2["Per Student Budget"],spending_bins,labels=labels).head()

In [ ]:
df2["Spending Ranges (Per Student)"] = pd.cut(df2["Per Student Budget"],spending_bins,labels=labels)
df2.head()

In [ ]:
spending_math_scores = df2.groupby(["Spending Ranges (Per Student)"])["Average maths score"].mean()
spending_reading_scores = df2.groupby(["Spending Ranges (Per Student)"])["Average reading score"].mean()
spending_passing_math = df2.groupby(["Spending Ranges (Per Student)"])["%Passing Maths"].mean()
spending_passing_reading = df2.groupby(["Spending Ranges (Per Student)"])["%Passing Reading"].mean()
overall_passing_spending = df2.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
#spending_summary
data_set = {"Average maths score": spending_math_scores, "Average reading score":spending_reading_scores, "%passing maths":spending_passing_math, "% passing reading": spending_passing_reading, "% overall passing":overall_passing_spending}
spending_summary_s = pd.DataFrame(data=data_set)
spending_summary = pd.DataFrame(data=data_set).style.format({
  "Average maths score": lambda val: f'{val:,.2f}', "Average reading score": lambda val: f'{val:,.2f}',"%passing maths": lambda val: f'{val:,.2f}',"% passing reading": lambda val: f'{val:,.2f}',"% overall passing": lambda val: f'{val:,.2f}'}) 
spending_summary

In [ ]:
size_bins = [0, 1000, 2000, 5000]
labels2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
pd.cut(df2["Total Students"],size_bins,labels=labels2).head()

In [ ]:
df2["School Size Ranges"] = pd.cut(df2["Total Students"],size_bins,labels=labels2)
df2.head()

In [ ]:
size_math_scores = df2.groupby(["School Size Ranges"])["Average maths score"].mean()
size_reading_scores = df2.groupby(["School Size Ranges"])["Average reading score"].mean()
size_passing_math = df2.groupby(["School Size Ranges"])["%Passing Maths"].mean()
size_passing_reading = df2.groupby(["School Size Ranges"])["%Passing Reading"].mean()
size_passing_overall = df2.groupby(["School Size Ranges"])["% Overall Passing"].mean()

In [ ]:
#size_summary
data_set_size = {"Average maths score": size_math_scores, "Average reading score":size_reading_scores, "%passing maths":size_passing_math, "% passing reading": size_passing_reading, "% overall passing":size_passing_overall}
size_summary_s = pd.DataFrame(data=data_set_size)
size_summary = pd.DataFrame(data=data_set_size).style.format({
  "Average maths score": lambda val: f'{val:,.2f}', "Average reading score": lambda val: f'{val:,.2f}',"%passing maths": lambda val: f'{val:,.2f}',"% passing reading": lambda val: f'{val:,.2f}',"% overall passing": lambda val: f'{val:,.2f}'}) 
size_summary

In [ ]:
type_math_scores = df2.groupby(["School Type"])["Average maths score"].mean()
type_reading_scores = df2.groupby(["School Type"])["Average reading score"].mean()
type_passing_math = df2.groupby(["School Type"])["%Passing Maths"].mean()
type_passing_reading = df2.groupby(["School Type"])["%Passing Reading"].mean()
type_passing_overall = df2.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
#type_summary
data_set_type = {"Average maths score": type_math_scores, "Average reading score":type_reading_scores, "%passing maths":type_passing_math, "% passing reading": type_passing_reading, "% overall passing":type_passing_overall}
type_summary_s = pd.DataFrame(data=data_set_type)
type_summary = pd.DataFrame(data=data_set_type).style.format({
  "Average maths score": lambda val: f'{val:,.2f}', "Average reading score": lambda val: f'{val:,.2f}',"%passing maths": lambda val: f'{val:,.2f}',"% passing reading": lambda val: f'{val:,.2f}',"% overall passing": lambda val: f'{val:,.2f}'}) 
type_summary

In [ ]:
#export txt
path="text.txt" 
with open(path, 'a') as f:
    area_summary_string = area_summary.to_string()
    f.write(area_summary_string)

In [ ]:
path="text.txt"
with open(path, 'a') as f:
    per_school_summary_string = per_school_summary.to_string()
    f.write(per_school_summary_string)

In [ ]:
path="text.txt"
with open(path, 'a') as f:
    top_schools_string = top_schools.to_string()
    f.write(top_schools_string)

In [ ]:
path="text.txt"
with open(path, 'a') as f:
    bottom_schools_string = bottom_schools.to_string()
    f.write(bottom_schools_string)

In [ ]:
path="text.txt" 
with open(path, 'a') as f:
    math_score_by_year_string = math_score_by_year.to_string()
    f.write(math_score_by_year_string)


In [ ]:
path="text.txt" 
with open(path, 'a') as f:
    reading_score_by_year_string = reading_score_by_year.to_string()
    f.write(reading_score_by_year_string)

In [ ]:
path="text.txt" 
with open(path, 'a') as f:
    spending_summary_string = spending_summary.to_string()
    f.write(spending_summary_string)

In [ ]:
path="text.txt" 
with open(path, 'a') as f:
   size_summary_string = size_summary.to_string()
   f.write(size_summary_string)

In [ ]:
path="text.txt" 
with open(path, 'a') as f:
   type_summary_string = type_summary.to_string()
   f.write(type_summary_string)